In [4]:
import pandas as pd
import re
import numpy as np
from src import functions
from src import correcciones
import datetime

In [5]:
cur = functions.connectToPs()

Connecting to Postgree...

<connection object at 0x0000022817FF9BF8; dsn: 'user=oxbigdatabi password=xxx dbname=bigdatabi host=bigdatabi.ckq5duwtue6h.eu-central-1.rds.amazonaws.com', closed: 0>


In [7]:
detalle = functions.detalle(cur)

In [17]:
detalle[detalle['assetid']=='02i0N00000Phb2IQAR'].T

,1836
segmento,3-NOTRIAL
Family,X SECURITY
product_name,X Security 50
companyname,MACON SA
sitename,GIRONA
cod_postal,08034
created_as_trial,false
purchase_date,2020-02-24 00:00:01
cancellation_date,NaT
service_date,NaT


In [4]:
df = functions.base(cur)

In [5]:
df_altas = functions.base_altas(df)

In [6]:
bill_df =  functions.billing(cur)

In [7]:
revenue_output = functions.revenue(bill_df)

In [8]:
bill_df[['factura']].groupby(by=[bill_df.mes]).sum()

,factura
mes,
2018-09,0.00
2018-10,17.20
2018-11,161.50
2018-12,842.21
2019-01,923.70
2019-02,2435.84
2019-03,3615.87
2019-04,4483.75
2019-05,5172.34


In [48]:
def billing(cur):
    cur.execute('''
                select bi.assetid, ca.product_name, ca.cif, ca.canal_venta, 
                substring(bi.dfrom,0,8) as mes, sum(bi.totalamount) as factura 
                from billing_details bi
                left join (
                            select * from cartera_xbo 
                            where process_date = (
                                                    select max(process_date) from cartera_xbo)) ca
                on bi.assetid = ca.assetid
                group by 1,2,3,4,5
            ''')
    df = pd.DataFrame(cur.fetchall())
    col_names = []
    for e in range(len(cur.description)):
        col_names.append(cur.description[e][0])
    df.rename(columns=dict(zip(list(range(33)), col_names)), inplace=True)
    df = df[df['cif'].notnull()]
    df['canal_venta'] = df.apply(lambda row: correcciones.partner(row), axis=1)
    df['canal_venta'] = df.canal_venta.apply(lambda x: functions.canal(x))
    df['Family'] = df.product_name.apply(lambda x: functions.getFamilyFrom(x))
    df['Velocity'] = df.product_name.apply(lambda x: functions.getVelocityFrom(x))
    df['isNeba'] = df.product_name.apply(lambda x: functions.isNeba(x))
    df['isMonosede'] = df.product_name.apply(lambda x: functions.isMonosede(x))
    df['product_name_ok'] = df.apply(lambda row: functions.getProductName(row), axis=1)
    cols = ['assetid','Family','product_name_ok', 'canal_venta','factura','mes']
    df = df[cols]
    return df

In [49]:
p = billing(cur)

In [46]:
p[p['mes']=='2020-02'].factura.sum()

17107.1

In [50]:
p

,assetid,Family,product_name_ok,canal_venta,factura,mes
20,02i0N000006M5MaQAK,X UCOM,Teamwork+PBX,WEB,0.00,2019-02
21,02i0N000006M5MaQAK,X UCOM,Teamwork+PBX,WEB,0.00,2019-03
22,02i0N000006M5MbQAK,X UCOM,Teamwork+PBX,WEB,0.00,2019-02
23,02i0N000006M5MbQAK,X UCOM,Teamwork+PBX,WEB,0.00,2019-03
24,02i0N000006M5McQAK,X UCOM,Teamwork+PBX,WEB,0.00,2019-02
...,...,...,...,...,...,...
8146,02i0N00000PSFbSQAX,X UCOM,Teamwork+PBX,DHO,8.12,2019-12
8147,02i0N00000PSFbSQAX,X UCOM,Teamwork+PBX,DHO,12.00,2020-01
8148,02i0N00000PSFbSQAX,X UCOM,Teamwork+PBX,DHO,12.00,2020-02
8149,02i0N00000PSFbTQAX,X UCOM,Teamwork+PBX,DHO,0.00,2019-12


## Ventas

In [6]:
functions.ass_vta_mes(df,2)
# a.iloc[a.index.get_level_values('Family') == 'X UCOM']

,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01,2020-02
canal_venta,,,,,,,,,,,,,,,,,
DHO,2,14,7,28,53,27,19,45,104,92,17,103,117,100,113,119,7
IT integrator,0,1,14,6,1,2,12,9,22,6,1,47,38,5,1,11,7
Venta Directa,0,5,1,0,0,1,2,0,1,1,0,4,6,2,15,6,8
WEB,1,4,12,1,2,24,8,8,11,6,5,12,12,3,2,0,0


In [8]:
functions.cli_altas_mes_agg(df_altas,tipo=1)

,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01,2020-02
canal_venta,,,,,,,,,,,,,,,,,
DHO,2,2,9,7,19,8,8,13,29,30,19,28,35,34,24,35,4
IT integrator,0,1,3,3,0,1,6,2,3,2,0,5,0,1,0,0,0
Venta Directa,0,1,0,1,0,0,1,0,1,1,0,2,2,2,1,4,1
WEB,1,1,6,2,1,5,3,6,3,3,2,5,7,2,0,0,0


In [16]:
x = df[df['Family']=='X UCOM']
x.to_csv('../x.csv ',encoding='CP1252')

In [4]:
# functions.ventas(df)

In [5]:
# functions.altas(df,df_altas )

In [5]:
functions.bajas(df,df_altas)

,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01,2020-02
index,,,,,,,,,,,,,,,,,
isBaja,0,0,1,3,0,2,6,8,5,4,5,8,10,7,7,8,1
DHO,0,0,0,1,0,2,4,4,2,3,2,3,5,4,6,8,0
IT integrator,0,0,0,0,0,0,0,2,1,0,0,3,0,0,0,0,0
WEB,0,0,1,2,0,0,2,2,2,1,3,2,5,3,1,0,1
DHO_X FIBER,0,0,0,0,0,1,0,0,0,2,1,1,1,0,1,2,0
DHO_X MUSIC F,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0
DHO_X PRIVACY,0,0,0,0,0,0,2,3,1,1,2,1,3,0,0,0,0
DHO_X PROTECTION F,0,0,0,0,0,0,0,2,3,0,1,1,1,2,2,6,0
DHO_X SECURITY,0,0,0,0,0,0,0,0,0,1,0,0,2,0,2,0,0


In [6]:
functions.cli_bajas_mes(df, 3)

2018-10  2018-11  2018-12  2019-01  2019-02  \
canal_venta   Family                                                        
DHO           X FIBER               0        0        0        0        0   
              X MUSIC F             0        0        0        1        0   
              X PRIVACY             0        0        0        0        0   
              X PROTECTION F        0        0        0        0        0   
              X SECURITY            0        0        0        0        0   
              X UCOM                0        0        0        1        1   
IT integrator X FIBER               0        0        0        0        0   
              X MUSIC F             0        0        0        0        0   
              X PROTECTION F        0        0        0        0        0   
              X SECURITY            0        0        0        0        0   
              X UCOM                0        0        0        0        0   
WEB           X FIBER               0        0        0        0        0   
              X MUSIC F             0        0        1        0        0   
              X PRIVACY             0        0        0        0        0   
              X PROTECTION F        0        0        0        2        0   
              X UCOM                0        0        0        0        1   

                              2019-03  2019-04  2019-05  2019-06  2019-07  \
canal_venta   Family                                                        
DHO           X FIBER               1        0        0        0        2   
              X MUSIC F             1        1        0        0        0   
              X PRIVACY             0        2        3        1        1   
              X PROTECTION F        0        0        2        3        0   
              X SECURITY            0        0        0        0        1   
              X UCOM                2        1        1        2        4   
IT integrator X FIBER               0        0        0        0        0   
              X MUSIC F             0        0        1        0        0   
              X PROTECTION F        0        1        0        1        0   
              X SECURITY            0        0        0        0        0   
              X UCOM                0        0        1        0        0   
WEB           X FIBER               1        0        0        1        0   
              X MUSIC F             0        1        1        0        0   
              X PRIVACY             0        1        0        0        0   
              X PROTECTION F        0        1        1        0        0   
              X UCOM                0        0        0        2        1   

                              2019-08  2019-09  2019-10  2019-11  2019-12  \
canal_venta   Family                                                        
DHO           X FIBER               1        1        1        0        1   
              X MUSIC F             0        0        0        0        0   
              X PRIVACY             2        1        3        0        0   
              X PROTECTION F        1        1        1        2        2   
              X SECURITY            0        0        2        0        2   
              X UCOM                3        2        4        3        5   
IT integrator X FIBER               0        2        0        0        1   
              X MUSIC F             0        1        0        0        0   
              X PROTECTION F        0        1        0        1        0   
              X SECURITY            0        0        0        0        0   
              X UCOM                0        0        0        0        0   
WEB           X FIBER               0        0        1        0        0   
              X MUSIC F             1        0        0        0        0   
              X PRIVACY             0        0        0        0        0   
              X PROTECTI

In [5]:
mig_df=functions.migBase(cur)

In [11]:
# mig_df

In [6]:
migin, migout = functions.mig_producto_mes(mig_df,df)

In [25]:
migin.reset_index(inplace=True)
migin['index'] = migin['canal_venta']+'_'+migin['product_name']
migin.drop(columns=['canal_venta','Family','product_name'],inplace=True)
migin.set_index(['index'],inplace=True)

In [7]:
migin

2018-10  2018-11  2018-12  2019-01  \
canal_venta   Family  product_name                                             
DHO           X FIBER X Fiber 100 - neba        0        0        0        0   
              X UCOM  Teamwork+PBX              0        0        0        0   
IT integrator X UCOM  Teamwork+PBX              0        0        0        0   

                                          2019-02  2019-03  2019-04  2019-05  \
canal_venta   Family  product_name                                             
DHO           X FIBER X Fiber 100 - neba        0        0        1        0   
              X UCOM  Teamwork+PBX              0        0        0       11   
IT integrator X UCOM  Teamwork+PBX              0        0        0        0   

                                          2019-06  2019-07  2019-08  2019-09  \
canal_venta   Family  product_name                                             
DHO           X FIBER X Fiber 100 - neba        0        0        0        0   
              X UCOM  Teamwork+PBX              0        2        1        1   
IT integrator X UCOM  Teamwork+PBX              1        0        0        0   

                                          2019-10  2019-11  2019-12  2020-01  \
canal_venta   Family  product_name                                             
DHO           X FIBER X Fiber 100 - neba        0        0        0        0   
              X UCOM  Teamwork+PBX              3        1        0        1   
IT integrator X UCOM  Teamwork+PBX              0        0        0        0   

                                          2020-02  
canal_venta   Family  product_name                 
DHO           X FIBER X Fiber 100 - neba        0  
              X UCOM  Teamwork+PBX              0  
IT integrator X UCOM  Teamwork+PBX              0

In [33]:
getVelocityFrom('20')

'Portable 20'